# Homework 1 : Unix Basics, Data Access, TSCC


1. [Git and Github](3_gitgoing.ipynb#) 
    - ~~To turn in: Nothing, but you have to do it to proceed~~
    - To turn in: Link to your forked `cheatsheets` repository
2. [Jupyter Notebook Setup](4_jupyter_setup.ipynb) 
    - To turn in: Nothing, but you have to do it to proceed
    - This has you clone the github repo for the class which is necessary for the next steps
3. [Fork and clone the BIOM262 GitHub repository, create `week01` branch](5_fork_clone_biom262-2016_repo.ipynb)
    - To turn in: Nothing, but you have to do it to proceed
4. [Which promoters does NFKB bind in the genome?](7_tf_binding_promoters.ipynb)
    - To turn in: Jupyter notebook of `7_tf_binding_promoters.ipynb` submitted as a Pull Request (PR) to the `biom262/biom262-2016` repository.
5. [Bringing it all together](8_bringing_it_all_together.ipynb)
    - To turn in: Link to a GitHub repository
    
## How to turn in your homework

### 1. Push your changes

```
git add weeks/week01/7_tf_binding_promoters.ipynb
git commit -m 'captivating commit message'
git push origin week01
```

### 2. On the GitHub website for your fork of the `biom262-2016` repo, click "New Pull Request":

![](images/new_pull_request.png "New Pull Request")

### 3. On the "Comparing changes" page, choose "compare: week01"

![](images/comparing_changes_branches.png "Choose a branch for comparing changes")

### 4. Click "Create Pull Request"

![](images/create_pull_request.png)

### 5. In the text of the pull request, add the following text, replacing the URLs with the relevant ones for the different sections.

```
- #1: [My cheatsheets repo](http://github.com/yourgithubusername/cheatsheets)
- #5: [Our TSCC script repo])(http://github.com/githubusername/repo_from_bringing_it_all_together)
```

### You can still make changes!

Even after you make the pull request, your new commits will still show up.

## FAQ + Tips and Tricks

### Logging in off-campus

For some reason TSCC isn't letting some people log on off-campus. To do that, you'll need to set up a VPN client following [these](https://blink.ucsd.edu/technology/network/connections/off-campus/VPN/index.html) instructions for the **VPN AnyConnect client**. Don't do EasyConnect because that will only send the traffic in your browser through UCSD, but you need all of the internetz through UCSD to be able to `ssh` and stuff.

### Can't log on to gitter

If you're using Safari on a Mac, then there's a [bug](https://github.com/drone/drone/issues/663#issuecomment-62459096) with the GitHub login for Gitter. Copy/paste the url into a different browser. If that doesn't work, right click the "sign in" link and paste into a different browser. If that doesn't work, set the other browser as the default and THEN click the link. Phew! Hopefully one of those works...

### "The authenticity of host XX can't be established"

Whenever you're logging into a server for the first time, you will ***always*** get a message from your computer because it's being extra-super-paranoid that to make sure you're not connecting to something new when you thought you were connecting to something old.

So, to solve the below problem, where you can't enter in "yes" or "no" because the process is running in the background due to the ampersand ("`&`").

```
BODMER2982:.ssh elmen$ ssh -NL 1066:localhost:1066 ucsd-train05@tscc-login2.sdsc.edu &
[1] 12131
BODMER2982:.ssh elmen$ The authenticity of host 'tscc-login2.sdsc.edu (132.249.107.88)' can't be established.
RSA key fingerprint is SHA256:Vai4AyMLYxtd9JzGrAFep819rpcfODzqz65DfmpbL8g.
Are you sure you want to continue connecting (yes/no)?
```

Do the same thing, but remove the ampersand ("`&`"). You should now be able to say "yes" (you'll have to type the whole thing out:

```
$ ssh -NL 1066:localhost:1066 ucsd-train05@tscc-login2.sdsc.edu
The authenticity of host 'tscc-login2.sdsc.edu (132.249.107.88)' can't be established.
RSA key fingerprint is SHA256:Vai4AyMLYxtd9JzGrAFep819rpcfODzqz65DfmpbL8g.
Are you sure you want to continue connecting (yes/no)? yes
Warning: Permanently added 'tscc-login2.sdsc.edu' (RSA) to the list of known hosts.
```
This is *completely normal*.

If you get a whole mess of `y`s:

```
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
```

Then close the tab and open a new one, and try again.


### What's up with this "home" directory anyway?

There's been some confusion about what is a "home" directory. On TSCC, there is a folder called `/home`, but this is *not* **your** home directory. If you look at `/home`:

```
$ ls /home
amuratov   ecwheele       henrikse  jnussbac    r4he          ucsd-train15  ysui
areinke    elvannostrand  hocks     jpg         rit001        ucsd-train17  yuh155
b1lake     escott         ihchu     kstandis    rmande        ucsd-train43  yul277
b2dong     esticca        j6yao     mhunt       serein        ucsd-train46  yul549
bcopeland  gmccord        jbrooks   mrierari    shsathe       ux451890      z4deng
chz022     gpratt         jenhan    nbutcher    tiliu         w6ye          z9wang
cnnguyen   grbala         jhayes    obotvinnik  tkc004        wbrandler     zhz004
davids4    grosanio       jic198    ongsp       ucsd-train01  yeo-lab       zix009
```

You'll see a bunch of usernames. The idea is that everyone has their own "house," so your "house" is `/home/ucds-train##`, and this is called your "home directory."

#### Tip: `cd` by itself takes you home

If you use `cd` by itself, it will take you back to your "house"! aka your "home directory." Check out the commands below:

```
[ucsd-train01@tscc-login2 ~]$ cd code/biom262-2016/weeks/week01
[ucsd-train01@tscc-login2 week01]$ pwd
/home/ucsd-train01/code/biom262-2016/weeks/week01
[ucsd-train01@tscc-login2 week01]$ cd
[ucsd-train01@tscc-login2 ~]$ pwd
/home/ucsd-train01
```

### Ahhh I did something I didn't want to do and want to get out!!

If you ran a program or command that you didn't want to, "Ctrl-`C`" is your friend. This will exit whatever program you're in and get you back to the command line. You can think of it as a "AHH I don't know what happened and I'm scared please let me out!! Panic panic!" button.

### Add shortcuts to make logging in easier

To help you log in to TSCC, for Mac: edit your `~/.bash_profile` (on Linux, edit your ``~/.bashrc`)

```
nano ~/.bash_profile
```

Add the lines below, replacing `ucsd-train##` with your train number.

```
alias tscc='ssh ucsd-train##@tscc-login#.sdsc.edu'
alias tunneltscc='ssh -NL ####:localhost:#### ucsd-train##@tscc-login#.sdsc.edu'
```

### Still getting asked for a password?

This seems to be a problem exclusive to Mac OS X "El Capitan" users. It turns out that `ssh-add` is not permanent and you have to do it every time you want to log in. To avoid having to explicitly write that command, instead we'll edit the `~/.bash_profile` on your laptop.

On your laptop, open up `~/.bash_profile` to edit using `nano`:

```
nano ~/.bash_profile
```

Now, add the following line:

```
ssh-add ~/.ssh/biom262_rsa.pri
```

Keep in mind that the file `~/.ssh/biom262_rsa.pri` must exist! If not, then you need to re-do [setting up your logins](http://nbviewer.ipython.org/github/biom262/biom262-2016/blob/master/weeks/week01/2_tscc_setup.ipynb#Set-up-your-logins).

#### "`Write failed: Broken pipe`"

If you see a "`Write failed: Broken pipe`" output in your laptop terminal, that means you've lost connection to the server. It's nothing to worry about. **Solution:** Re-run your `ssh` command (either logging in to TSCC or doing the tunneling) and you'll be all set.

### Anaconda installation issues

#### Already have python error
If you're getting an error that you already have Python around that looks like this:

```
WARNING:
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Anaconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Anaconda3: /home/ucsd-train46/anaconda3
Do you wish the installer to prepend the Anaconda3 install location
to PATH in your /home/ucsd-train46/.bashrc ? [yes|no]
[no] >>> 
```
Type "`yes`"! Do not press "enter" because then it will do the default which is "`no`". If you pressed "enter", that's okay, move on to the next fix

#### Make sure the Anaconda installer *actually* added lines to your `~/.bashrc`

Use `nano` to look at your `~/.bashrc` and edit it just in case:

```
$ nano ~/.bashrc
```

If there is no line that looks like the one below (replacing `##` with your train number), then add it to the `~/.bashrc` file.

```
# added by Anaconda3 2.4.1 installer
export PATH="/home/ucsd-train##/anaconda3/bin:$PATH"
```

Save and close the file.

#### "Cannot connect to https://conda.anaconda.org/"

If you're having issues connecting to the `https://conda.anaconda.org/r` channel, do the `conda install` command with everything except the `https://conda.anaconda.org/`, as below:

```
conda install --channel r r r-irkernel seaborn
```


### Jupyter problems

#### Output after `jupyter notebook`

You will see a number get output from the `jupyter notebook --no-browser --port #### &` command before you see the rest. That is totally normal and expected.

```
$ jupyter notebook --no-browser --port #### &
[1] 12583
(biom262)[ucsd-train12@tscc-login2 ~]$ [I 13:23:05.786 NotebookApp] Writing notebook server cookie secret to /home/ucsd-train12/.local/share/jupyter/runtime/notebook_cookie_secret
[I 13:23:06.291 NotebookApp] Serving notebooks from local directory: /home/ucsd-train12
[I 13:23:06.291 NotebookApp] 0 active kernels
[I 13:23:06.291 NotebookApp] The IPython Notebook is running at: http://localhost:7788/
[I 13:23:06.291 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
```

#### Bug in instructions - you must use `--no-browser` flag

At some point the instructions on the website didn't have the `--no-browser` flag for `jupyter`. Make sure that your jupyter command includes this, and thus looks like:

```
jupyter notebook --no-browser --port #### &
```

If you don't do that, you'll get this big weird screen:

![](images/jupyter_minus_no_browser_flag.png)

#### "The port #### is already in use, trying another random port"

This can happen if you ran `jupyter notebook` and killed it but wanted to run it again. This is because even though the `jupyter notebook` process died, there's still a "[zombie](https://en.wikipedia.org/wiki/Zombie_process)" (real computer term) process running. To kill it, do:

```
$ ps
  PID TTY          TIME CMD
 6593 pts/44   00:00:00 bash
12583 pts/44   00:00:02 jupyter-noteboo
12592 pts/44   00:00:00 links
14198 pts/44   00:00:01 jupyter-noteboo
14208 pts/44   00:00:00 ps
```

Look at the number that's at the beginning of the line for `jupyter-notebook` and then use `kill -9` to stop them ("`kill`" stops the process and there's flags of the extremeness of the killing, where "`-0`" is pretty chill and "`-9`" means the meanest form like 1st degree premeditated murder):

```
$ kill -9 12583
$ kill -9 14198
```

#### "No such file or directory"

If you get this error:

```
[ucsd-train21@tscc-login1 ~]$ jupyter notebook —no-browser —port 1224
[C 18:12:26.584 NotebookApp] No such file or directory: /home/ucsd-train21/—no-browser
```

That means that you forgot the extra dashes for `--no-browser` and `--port`. Try again with:

```
jupyter notebook —-no-browser —-port ####
```


### Tunneling Errors

#### `The authenticity of host blah blah can't be established`

This process is saying it's not sure about the computer you're connecting to, which may happen because before you logged on to `tscc.sdsc.edu` and now you're using `tscc-login#.sdsc.edu`, and this is specifying a different computer and now your laptop is being super paranoid and double checking with you again.

To fix this, remove the "`&`" (called an "ampersand" or "and symbol") from the tunneling command. Thus, your tunneling command should look like:

```
ssh -NL ####:localhost:#### username@tscc-login#.sdsc.edu
```

The first time you connect, you'll get output like this. Say "yes" that you want to continue connecting. Computers are paranoid schizophrenic and freak out any time something new is happening.

```
MacBook:~ benlewis$ ssh -NL 1225:localhost:1225 ucsd-train21@tscc-login1.sdsc.edu
The authenticity of host 'tscc-login1.sdsc.edu (132.249.107.90)' can't be established.
RSA key fingerprint is ee:40:c3:c6:19:03:9d:29:23:e6:ee:82:80:02:87:9b.
Are you sure you want to continue connecting (yes/no)? y
Please type 'yes' or 'no': yes
Warning: Permanently added 'tscc-login1.sdsc.edu' (RSA) to the list of known hosts.
```



#### "`bind: Address already in use`"

If you try to do the tunneling and you see this kind of output:
```
MacBook:~ benlewis$ ssh -NL 1224:localhost:1224 ucsd-train21@tscc-login2.sdsc.edu
bind: Address already in use
channel_setup_fwd_listener: cannot listen to port: 1224
Could not request local forwarding
```

That happens when you've run multiple of the `ssh -NL ####:localhost:#### ucsd-train##@tscc-login#.sdsc.edu` commands, and your computer is getting confused because you're telling it to do multiple things with the same port number.

**Solution:** Close the tab and open a new one - start fresh! Try doing the `ssh -NL ...` command again.


#### Tunneling: Windows

To get Jupyter notebook on your computer, you'll need to set up a "SSH Tunnel" that "listens" to that particular port, and thus gets the Jupyter notebook from TSCC. 

##### Step 1: Create a new Putty session

So that you only need to have one Putty session open, we'll make a new TSCC Session. Create one with `ucsd-train##@tscc-login#.sdsc.edu`, and call the session "TSCC Jupyter"

![](images/putty_tscc_jupyter_step1.png)

##### Step 2: Add your private key and allow forwarding

Go to "Connection > SSH > Auth." Click the checkbox next to "Allow agent forwarding" and add the "Putty Private Key" that you created with the `biom262_rsa` file.

![](images/putty_tscc_jupyter_step2.png)

##### Step 3: Add a tunnel

Go to "Connnection > SSH > Tunnels." Then:

1. Click the checkbox next to "Local ports accept connections from other ports"
2. Add your `####` for your source port
3. Add `localhost:####` for your Destination
4. Click "Local"
5. Click "Add"

![](images/putty_tscc_jupyter_step3.png)

You should now see this:

![](images/putty_tscc_jupyter_step3_added.png)

##### Step 4: Save your settings!

So you don't have to do this every time... Save your settings! Go all the way back to the "Session" window and click "Save"

![](images/putty_tscc_jupyter_step4.png)

##### Step 5: Log in! You may see this message below. It's totally normal

![](images/putty_tscc_jupyter_step5.png)

##### Step 6: Run your `jupyter notebook` command

You'll see this kind of output:

![](images/putty_tscc_jupyter_step6.png)


And now you can move on to the next step to view the notebooks! Go to `http://localhost:####` in your browser (Chrome, Firefox, IE)


### NFKB Binding sites - updated homework

Due to a difference in the implementation of `wc` on Mac vs TSCC's Linux, all the `assert` statements will fail which makes me a sad panda. So, the homework has been updated, as has the "Random 10 lines" bug that wasn't actually showing 10 random lines. To get the updated homework while saving your work, follow these steps.

I've written them all out explicitly to hopefully avoid most issues ([here](http://stackoverflow.com/questions/18734739/using-ipython-notebooks-under-version-control) is a big ole discussion about how Jupyter/IPython notebooks are hard to keep under version control because of their werid format) but again if you run into anything, come chat on the [gitter](https://gitter.im/biom262/biom262-2016).


#### Step 1: Go to your week01 directory

```
$ cd code/biom262-2016/weeks/week01
```

#### Step 2: Do `git status`


This is to check two things:
 
1. What branch you're on (should be `week01` but we can fix that if it's not the case)
2. What files are modified

```
$ git status
# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   7_tf_binding_promoters.ipynb
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	data/gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
#	data/tf.nkfb.bed
#	tf.nfkb.bed
no changes added to commit (use "git add" and/or "git commit -a")
```

The reason we're doing branching is because it mimics the way that you add features or new code in a way that keeps the current version pristine.

##### If not `On branch week01`

Whether or not you've commited changes, you can move all your changes to a new branch with:

```
git checkout -b week01
```

Now when you say `git status` it should show that you're "On branch week01":

```
$ git status
# On branch week01
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   7_tf_binding_promoters.ipynb
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	data/gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
#	data/tf.nkfb.bed
no changes added to commit (use "git add" and/or "git commit -a")

```

#### *Do not skip this step!* Step 3: Remove unnecessary things from your notebook

This step is important because it minimizes the differences between the notebook  that you've been working so hard on and the updated one that you're getting. If you don't do this ... you will have 100x more headaches, guaranteed.

Again, ***Do not skip this step***.

If you actually look at a Jupyter notebook, like the raw values of the file, you'll see a bunch of curly braces "`{}`" and other things that are there:

```
$ head -n 20 7_tf_binding_promoters.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "deletable": false,
    "nbgrader": {
     "checksum": "d4ff841edbfee192d053256b53bf2b90",
     "grade": false,
     "grade_id": "01_setup",
     "locked": true,
     "solution": false
    }
   },
   "source": [
    "# Homework 1: File formats and TF binding sites\n",
    "\n",
    "By the end of this homework you will...\n",
    "\n",
    "* Know what common genome annotation formats are (BED, GTF)\n",
```

That doesn't seem so bad. Why would this be a problem? Well, let's look at the 10 lines of text before ("`-B`") and after ("`-A`") the line `print(exercise1)`:

```
$ grep -A 10 -B 10 'print(exercise1)' 7_tf_binding_promoters.ipynb
      "chr22\t51001317\t51001593\tNFKB\t380\t.\t51001317\t51001593\t0\t1\t276,\t0,\n",
      "chr22\t51021040\t51021316\tNFKB\t484\t.\t51021040\t51021316\t0\t1\t276,\t0,\n",
      "chr22\t51021478\t51021754\tNFKB\t363\t.\t51021478\t51021754\t0\t1\t276,\t0,\n",
      "chr22\t51058900\t51059230\tNFKB\t498\t.\t51058900\t51059230\t0\t1\t330,\t0,\n",
      "chr22\t51060107\t51060383\tNFKB\t354\t.\t51060107\t51060383\t0\t1\t276,\t0,\n",
      "\n"
     ]
    }
   ],
   "source": [
    "print(exercise1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {
    "collapsed": false,
    "deletable": false,
    "nbgrader": {
     "checksum": "43c58a60155a8bc91e93d15bcfa69534",
```
 
Well, we have the output of the previous cell, and the `"execution_count": 29,` line which will be different in the updated notebook we're getting.

##### Install `nbstripout`

To remove the outputs and execution count and other stuff we don't need for the notebook, we need to install this utility called `nbstripout`. First, we'll make a `$HOME/bin` directory (in case you want to know: the `bin` name comes from the word "binaries" aka [historically] things that are compiled from raw C code to something runable, but with Python the raw stuff is runable), then save this file from github to that directory.

```
$ mkdir $HOME/bin
$ wget --output-document $HOME/bin/nbstripout  https://raw.githubusercontent.com/kynan/nbstripout/master/nbstripout.py
```

Try running `nbstripout`. It shouldn't work.

```
$ nbstripout
-bash: nbstripout: command not found
```

Similarly to how the `conda` command didn't run at first, we have to tell our command line that this cool new directory exists with programs that we can run. So, edit your `~/.bashrc`:

```
nano ~/.bashrc
```

And add this line to the end:

```
export PATH=$HOME/bin:$PATH
```

Your `~/.bashrc` should now look like this:

```
$ cat ~/.bashrc
# .bashrc

# Source global definitions
if [ -f /etc/bashrc ]; then
	. /etc/bashrc
fi

# User specific aliases and functions

# added by Anaconda3 2.4.1 installer
export PATH="/home/ucsd-train01/anaconda3/bin:$PATH"
export PATH=$HOME/bin:$PATH
```

Now source your `~/.bashrc` as before:

```
$ source ~/.bashrc
```

And try to run `nbstripout`. If you get a "Permission denied" error, you'll need to make that file "e***x***ecutable", i.e. runable from the command line. This is protection so someone couldn't write a function like `ls` that's actually malicious and deletes all your work, without being executable.

```
$ nbstripout
-bash: /home/ucsd-train01/bin/nbstripout: Permission denied
```

To make that file runable, use `chmod`:
```
$ chmod ugo+x ~/bin/nbstripout
```

Now if you try `nbstripout` it'll hang for a bit:

```
$ nbstripout

```

Do Ctrl-C or Ctrl-Z (I always forget which one and end up button mashing on Ctrl-Z, Ctrl-X, and Ctrl-C) to get out of it.

```
$ nbstripout
^Z
[7]+  Stopped                 nbstripout
```

##### Run the `nbstripout` command

Now that we have our `nbstripout` command at the ready, we can run it! You don't need to run the `cd` command if you're already in the `week01` directory.

```
$ cd ~/code/biom262-2016/weeks/week01/
$ nbstripout 7_tf_binding_promoters.ipynb
```


##### Verify for yourself that `nbstripout` did what it said

Check out the 10 lines before and after `print(exercise1)` now. Notice that all the output above is removed and the `execution_count` is set to `null`. This is important because these now match the updated `biom262/biom262-2016` version of the notebook. 
```
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "print(exercise1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": false,
    "deletable": false,
    "nbgrader": {
     "checksum": "96d21342bdcb1b2153be8df3ed6bc4f2",
```


#### Step 4: Commit your work

Use `git add`  and `git commit` to add your work so far.

```
git add 7_tf_binding_promoters.ipynb
git commit -m "add work up to exercise 2"
```

#### Step 4: Check your remote repositories

Use the command `git remote -v` (the `-v` is for "verbose" as in "tell me everything you know") to see which github repositories this git folder knows about. If your output looks like the one below, you're golden and can move on to step 4. If not, follow the instructions below

```
$ git remote -v
origin	https://github.com/olgabot/biom262-2016.git (fetch)
origin	https://github.com/olgabot/biom262-2016.git (push)
upstream	https://github.com/biom262/biom262-2016.git (fetch)
upstream	https://github.com/biom262/biom262-2016.git (push)
```

##### Add the `biom262/biom262-2016` repository as `upstream` and set yours as `origin`


If your ouptut looks like this, never fear:

```
$ git remote -v
origin	https://github.com/biom262/biom262-2016.git (fetch)
origin	https://github.com/biom262/biom262-2016.git (push)
```

It's likely you haven't forked the repository to your own username, so go do that now. Click [here](https://github.com/biom262/biom262-2016) and then click "Fork." This will make your own personal copy of the repository.

Next do these commands:

```
$ git remote add upstream https://github.com/biom262/biom262-2016.git
$ git remote set-url origin  https://github.com/yourgithubusername/biom262-2016.git
```
And make sure they were actually added and that your `git remote -v` output looks like this:

```
$ git remote -v
origin	https://github.com/olgabot/biom262-2016.git (fetch)
origin	https://github.com/olgabot/biom262-2016.git (push)
upstream	https://github.com/biom262/biom262-2016.git (fetch)
upstream	https://github.com/biom262/biom262-2016.git (push)
```

#### Step 5: Get the changes from the official repo (but don't add them yet) with `git fetch`

Get the changes that have been made to the homework with `git fetch upstream` which will specifically get the changes from `biom262/biom262-2016`.

```
$ git fetch upstream
remote: Counting objects: 24, done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.
From https://github.com/biom262/biom262-2016
 * [new branch]      gh-pages   -> upstream/gh-pages
 * [new branch]      master     -> upstream/master
```


#### Step 6: Merge with the `upstream/master` version

Okay, we are now ready to merge with the `upstream/master` version.

```
$ git merge upstream/master
Auto-merging weeks/week01/7_tf_binding_promoters.ipynb
CONFLICT (content): Merge conflict in weeks/week01/7_tf_binding_promoters.ipynb
Automatic merge failed; fix conflicts and then commit the result.
```
This `CONFLICT` is expected. Git is teaching you the valuable life lesson of learning to deal with conflict. Press your "Don't Panic" button and we'll fix the conflicts together. You won't be able to open the notebook in Jupyter while this is happening or you'll get an error.

#### Step 7: Fix merge conflicts

We're doing a deep dive and we're going to edit the `ipynb` file to fix the merge conflicts, as you [already learned](http://swcarpentry.github.io/git-novice/09-conflict.html) about from the Software Carpentry git intro.

```
$ nano 7_tf_binding_promoters.ipynb
```

Use the arrow keys to scroll downwards until you see the first conflict, as marked by the `<<<<<<< HEAD` line:

```
   "outputs": [],
   "source": [
    "%%bash --out exercise1\n",
<<<<<<< HEAD
    "look at me I am where your answer should go but I am not a real answer",
=======
    "# YOUR CODE HERE\\n\n",
>>>>>>> upstream/master
    "\n",
    "wc -l tf.nfkb.bed\n",
    "echo '--- First 10 lines ---'\n",
    "head tf.nfkb.bed\n",
    "echo '--- Random 10 lines ---'\n",
    "awk -v seed=907 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}'  tf.nfkb.bed | head\n",
    "echo '--- Last 10 lines ---'\n",
    "tail tf.nfkb.bed"
   ]
```

Delete these lines:

```
<<<<<<< HEAD
```

And

```
=======
    "# YOUR CODE HERE\\n\n",
>>>>>>> upstream/master
```

Your file should now look like this:

```
   "outputs": [],
   "source": [
    "%%bash --out exercise1\n",
    "look at me I am where your answer should go but I am not a real answer",
    "\n",
    "wc -l tf.nfkb.bed\n",
    "echo '--- First 10 lines ---'\n",
    "head tf.nfkb.bed\n",
    "echo '--- Random 10 lines ---'\n",
    "awk -v seed=907 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}'  tf.nfkb.bed | head\n",
    "echo '--- Last 10 lines ---'\n",
    "tail tf.nfkb.bed"
   ]
  },
```

Continue making these changes, removing the git nonsense of `<<<<<<< HEAD` and keeping your answer.

#### Step 8: Save your changes!

Check your work with `git status`:

```
$ git status
# On branch week01
# Changes to be committed:
#
#	modified:   4_jupyter_setup.ipynb
#	modified:   homework.ipynb
#	new file:   images/putty_tscc_jupyter_step1.png
#	new file:   images/putty_tscc_jupyter_step2.png
#	new file:   images/putty_tscc_jupyter_step3.png
#	new file:   images/putty_tscc_jupyter_step3_added.png
#	new file:   images/putty_tscc_jupyter_step4.png
#	new file:   images/putty_tscc_jupyter_step5.png
#	new file:   images/putty_tscc_jupyter_step6.png
#
# Unmerged paths:
#   (use "git add/rm <file>..." as appropriate to mark resolution)
#
#	both modified:      7_tf_binding_promoters.ipynb
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	data/gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
#	data/tf.nkfb.bed
```

Now add and commit your file:

```
$ git add 7_tf_binding_promoters.ipynb 
$ git commit -m 'merged updates from upstream/master'
[week01 7c25c24] merged updates from upstream/master
```

Check the status:

```
$ git status
# On branch week01
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	data/gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
#	data/tf.nkfb.bed
nothing added to commit but untracked files present (use "git add" to track)
```

#### Step 9: ...

There is no step 9! You're done! Yay! Now open your Jupyter notebooks and continue working :)


### `less` in Jupyter notebook

You won't be able to use `less` or anything else that you scroll through or need input from (`man`, `nano`) in Jupyter notebook - it will hang and you wont' be able to enter anything. In the Jupyter file menu, click "Kernel > Interrupt" to kill it, and stick to using `less`/`man`/`nano` in the vanilla terminal.